In [1]:
from random import choice, sample, randrange, random, shuffle
from string import ascii_lowercase
import numpy as np
from itertools import permutations

### Defining the fitness function

$fitness(w) = \sum^{nºguesses}_{i=1}(|correct(g_i,w) - c_i|+|misplaced(g_i,w) - m_i|)$

Where $w$ is a word for which we will evaluate the fitness, $g_i$ is the guess played on turn $i$, $c_i$ is the number of correct values obtained in the guess $i$, $correct(g,r)$ return the number of correct positions in a Wordle game provided that $g$ is the attempt and $w$ is the secret word, $misplaced(g,w)$ returns the number of misplaced letters given that $g$ is the attempt and $w$ is the secret word, $m_i$ is the number of misplaced letters obtained in the attempt $i$. 

In [2]:
def find(s, ch):
    return [i for i, ltr in enumerate(s) if ltr == ch]

def check_words(input_word ,target):
    input_word = input_word.lower()
    target = target.lower()
    sequence = ['⬜']*5
    selected = [False]*5
    for i in range(0,5):
        if input_word[i] == target[i]:
            sequence[i] = '🟩'
            selected[i] = True
    for i in range(0,5):
        if input_word[i] != target[i]:
            indexes = find(target, input_word[i])
            for index in indexes:
                if not selected[index]:
                    selected[index] = True
                    sequence[i] = '🟨'
                    break
    aux = ""
    return aux.join(sequence)

In [3]:
game_state = [('Hello', '🟩🟩🟨🟨🟨')]

In [4]:
def fitness(word, game_state):
    sum = 0
    for guess, result in game_state:
        match = result.count('🟩')
        misplaced =  result.count('🟨')
        result = check_words(guess, word)
        pos_match = result.count('🟩')
        pos_misplaced = result.count('🟨')
        sum += abs(pos_match - match) + abs(pos_misplaced - misplaced) + abs(match + misplaced - pos_match - pos_misplaced)
    return sum

### Defining mutation
Mutation occurs when a random letter of the word is changed.

In [5]:
def mutate(word, prob=0.03):
    if random() < prob:
        word = list(word)
        word[randrange(len(word))] = choice(ascii_lowercase)
        word = ''.join(word)
    return word

### Defining permutation
Permutation occurs when two letters from a word are switched

In [6]:
def permute(word, prob=0.03):
    if random() < prob:
        idx1, idx2 = sample(range(len(word)), 2)
        word = list(word)
        word[idx1], word[idx2] = word[idx2], word[idx1]
        word = ''.join(word)
    return word

### Defining inversion
Inversion occurs when two positions are randomly selected and the sequence of letters between is inverted.

In [7]:
def invert(word, prob=0.02):
    if random() < prob:
        idx1, idx2 = sample(range(len(word)), 2)
        idx1, idx2 = min(idx1,idx2), max(idx1,idx2)
        word = list(word)
        word[idx1:idx2] = reversed(word[idx1:idx2])
        word = ''.join(word)
    return word

### Defining crossover

In [8]:
def crossover(parent1, parent2, prob=0.5):
    child1, child2 = parent1, parent2
    if random() < prob:
        split = randrange(1, len(parent1)-1)
        child1 = parent1[:split] + parent2[split:]
        child2 = parent2[:split] + parent1[split:]
    return [child1, child2]

### Defining selection
We will use tournament based selection.

In [9]:
def selection(fitnesses, population, k=20, keep_size=True):
    k = min(k, len(population))
    def random_tournament():
        selected_idx = randrange(len(population))
        for idx in sample(range(len(population)), k=k):
            if fitnesses[idx] > fitnesses[selected_idx]:
                selected_idx = idx
        return population[selected_idx]
    return [random_tournament() for _ in range(len(population))]

### Creating the model

In [10]:
def wordle_genetic(game_state, guess_count, pop_size, possible_guesses, max_gen, tour_size, crossover_prob,
                   mutate_prob, permute_prob, invert_prob):
    
    if guess_count == 0:
        possible_guesses.remove('about')
        return "ABOUT"

    # create population
    sample_size = min(pop_size, len(possible_guesses))
    population = sample(possible_guesses, sample_size)
    eligible_words = set()
    generation = 0

    # do genetic iterations
    while generation < max_gen:
        # selection
        fitnesses = [fitness(p, game_state) for p in population]
        selected = selection(fitnesses, population, k=tour_size)

        # new generation
        new_pop = []
        for p1, p2 in zip(selected[0::2], selected[1::2]):
            for c in crossover(p1, p2, prob=crossover_prob):
                c = mutate(c, prob=mutate_prob)
                c = permute(c, prob=permute_prob)
                c = invert(c, prob=invert_prob)

                if (c in eligible_words) or (c not in possible_guesses):
                    new_pop.append(choice(tuple(possible_guesses)))
                else:
                    new_pop.append(c)

        population = new_pop
        eligible_words.update(population)

        generation += 1

    # choose word in eligible_words with maximum
    best_word = min(eligible_words, key=lambda x: fitness(x, game_state))
    possible_guesses.remove(best_word)
    return best_word

In [11]:
valid_inputs = []
with open("valid-wordle-words.txt") as f:
    valid_inputs = f.read().splitlines()

In [12]:
target = 'abbey'
eligible_words = set(valid_inputs)
game_state = []
word = wordle_genetic(game_state = game_state, guess_count = 0, pop_size = 150, 
                                     possible_guesses = eligible_words, max_gen = 100, tour_size = 40,
                                    crossover_prob = 0.5,  mutate_prob = 0.03, permute_prob = 0.03, invert_prob = 0.02)
check_words(word, target)
game_state.append((word, check_words(word, target)))
print(str(game_state[-1][1]) + " " +game_state[-1][0])
i = 1
while game_state[-1][1] != "🟩🟩🟩🟩🟩" and i <= 5:
    word = wordle_genetic(game_state = game_state, guess_count = i, pop_size = 150, 
                                     possible_guesses = eligible_words, max_gen = 100, tour_size = 40,
                                    crossover_prob = 0.5,  mutate_prob = 0.03, permute_prob = 0.03, invert_prob = 0.02)
    check_words(word, target)
    game_state.append((word, check_words(word, target)))
    print(str(game_state[-1][1]) + " " + game_state[-1][0])
    i += 1
if game_state[-1][1] == "🟩🟩🟩🟩🟩":
    print("Ended with a win in " + str(i) + " attempts")
else:
    print('Ended with a loss')

🟩🟩⬜⬜⬜ ABOUT


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_8404\3451594227.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  population = sample(possible_guesses, sample_size)


⬜⬜⬜⬜⬜ swopt
🟩🟩⬜🟨⬜ abaya
🟩🟩🟨⬜⬜ abear
🟩🟩🟩🟩🟩 abbey
Ended with a win in 5 attempts


In [13]:
def tryout(target):
    eligible_words = set(valid_inputs)
    game_state = []
    word = wordle_genetic(game_state = game_state, guess_count = 0, pop_size = 150, 
                                         possible_guesses = eligible_words, max_gen = 100, tour_size = 40,
                                        crossover_prob = 0.5,  mutate_prob = 0.03, permute_prob = 0.03, invert_prob = 0.02)
    check_words(word, target)
    game_state.append((word, check_words(word, target)))
    print(str(game_state[-1][1]) + " " +game_state[-1][0])
    i = 1
    while game_state[-1][1] != "🟩🟩🟩🟩🟩" and i <= 5:
        word = wordle_genetic(game_state = game_state, guess_count = i, pop_size = 150, 
                                         possible_guesses = eligible_words, max_gen = 100, tour_size = 40,
                                        crossover_prob = 0.5,  mutate_prob = 0.03, permute_prob = 0.03, invert_prob = 0.02)
        check_words(word, target)
        game_state.append((word, check_words(word, target)))
        print(str(game_state[-1][1]) + " " + game_state[-1][0])
        i += 1
    if game_state[-1][1] == "🟩🟩🟩🟩🟩":
        print("Ended with a win in " + str(i) + " attempts")
    else:
        print("Ended with a loss")

In [14]:
tryout('hello')

⬜⬜🟨⬜⬜ ABOUT


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_8404\3451594227.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  population = sample(possible_guesses, sample_size)


⬜⬜🟩⬜⬜ salic
⬜🟨⬜⬜🟨 foxie
⬜⬜⬜🟨⬜ wafer
⬜🟩⬜🟨⬜ sekos
🟩🟩🟩🟩🟩 hello
Ended with a win in 6 attempts


In [15]:
tryout('where')

⬜⬜⬜⬜⬜ ABOUT


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_8404\3451594227.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  population = sample(possible_guesses, sample_size)


⬜⬜⬜⬜⬜ vinic
⬜⬜⬜🟨🟨 plyer
⬜🟨🟨⬜🟩 merge
⬜🟨🟩🟩⬜ jeers
⬜🟨🟩⬜⬜ dregs
Ended with a loss


### Modifications using the data
#### Choosing the first attempt

In [16]:
frecs = np.load('new_word_frequency_per_cluster.npy', allow_pickle = True)
frec0 = frecs[0]
g0_sorted = sorted(frec0.items(), key=lambda x:x[1], reverse = True)
g0_top = g0_sorted[:30]
g0_top = np.asarray(g0_top)

In [17]:
top_vals = g0_top[:,1].astype(float)
top_vals = top_vals/top_vals.sum()
top_labels = g0_top[:,0]

In [18]:
def first_word(labels, vals):
    return labels[np.random.choice(len(labels), p = vals)]

#### Guesses based on freq

In [19]:
def wordle_genetic(game_state, guess_count, pop_size, dict_frec, max_gen, tour_size, crossover_prob,
                   mutate_prob, permute_prob, invert_prob, top_vals, top_labels):
    
    if guess_count == 0:
        guess = first_word(top_labels, top_vals)
        dict_frec.pop(guess)
        return guess
    
    possible_guesses = np.asarray(list(dict_frec.keys()))
    frecs = np.asarray(list(dict_frec.values()))

    # create population
    sample_size = min(pop_size, len(possible_guesses))
    population = np.random.choice(a = possible_guesses, size = sample_size , replace = False, p = frecs)
    eligible_words = set()
    generation = 0

    # do genetic iterations
    while generation < max_gen:
        # selection
        fitnesses = [fitness(p, game_state) for p in population]
        selected = selection(fitnesses, population, k=tour_size)

        # new generation
        new_pop = []
        for p1, p2 in zip(selected[0::2], selected[1::2]):
            for c in crossover(p1, p2, prob=crossover_prob):
                c = mutate(c, prob=mutate_prob)
                c = permute(c, prob=permute_prob)
                c = invert(c, prob=invert_prob)

                if (c in eligible_words) or (c not in possible_guesses):
                    new_pop.append(np.random.choice(a = possible_guesses, p = frecs))
                else:
                    new_pop.append(c)

        population = new_pop
        eligible_words.update(population)

        generation += 1

    # choose word in eligible_words with maximum
    best_word = min(eligible_words, key=lambda x: fitness(x, game_state))
    dict_frec.pop(best_word)
    return best_word

In [20]:
def update_dict(dict_frec):
    frecs = np.asarray(list(dict_frec.values()))
    frecs = frecs/frecs.sum()
    return dict(zip(dict_frec.keys(), frecs))

In [21]:
def tryout(target, frecs):
    #Prep sets
    frec0 = frecs[0]
    dict_frec = frec0
    g0_sorted = sorted(frec0.items(), key=lambda x:x[1], reverse = True)
    g0_top = g0_sorted[:30]
    g0_top = np.asarray(g0_top)
    top_vals = g0_top[:,1].astype(float)
    top_vals = top_vals/top_vals.sum()
    top_labels = g0_top[:,0]
    game_state = []
    frecs = np.asarray(list(dict_frec.values()))
    frecs = (frecs + abs(min(frecs)))
    frecs = frecs/frecs.sum()
    dict_frec = dict(zip(dict_frec.keys(), frecs))
    word = wordle_genetic(game_state = game_state, guess_count = 0, pop_size = 150, 
                                         dict_frec = dict_frec, max_gen = 100, tour_size = 40,
                                        crossover_prob = 0.5,  mutate_prob = 0.03, permute_prob = 0.03, invert_prob = 0.02,
                                         top_vals = top_vals, top_labels = top_labels)
    check_words(word, target)
    game_state.append((word, check_words(word, target)))
    print(str(game_state[-1][1]) + " " +game_state[-1][0])
    i = 1
    while game_state[-1][1] != "🟩🟩🟩🟩🟩" and i <= 5:
        dict_frec = update_dict(dict_frec)
        word = wordle_genetic(game_state = game_state, guess_count = i, pop_size = 150, 
                                         dict_frec = dict_frec, max_gen = 100, tour_size = 40,
                                        crossover_prob = 0.5,  mutate_prob = 0.03, permute_prob = 0.03, invert_prob = 0.02,
                                         top_vals = top_vals, top_labels = top_labels)
        check_words(word, target)
        game_state.append((word, check_words(word, target)))
        print(str(game_state[-1][1]) + " " + game_state[-1][0])
        i += 1
    if game_state[-1][1] == "🟩🟩🟩🟩🟩":
        print("Ended with a win in " + str(i) + " attempts")
    else:
        print("Ended with a loss")

In [22]:
tryout('hello', frecs)

⬜🟨🟨⬜⬜ there
⬜⬜⬜🟨⬜ ruder
⬜🟩⬜⬜⬜ nests
⬜🟩🟩⬜🟨 felch
🟩🟩🟩🟩🟩 hello
Ended with a win in 5 attempts


In [23]:
tryout('where', frecs)

🟩🟩⬜⬜⬜ which
⬜🟨⬜⬜🟨 felch
⬜🟩⬜⬜⬜ thiol
🟩🟩⬜🟩⬜ wharf
🟩🟩🟩🟩🟩 where
Ended with a win in 5 attempts


In [24]:
tryout('allow', frecs)

🟩⬜🟨⬜⬜ about
⬜🟨⬜⬜🟨 pooja
🟩⬜⬜🟩⬜ ammon
🟩🟩🟩🟩🟩 allow
Ended with a win in 4 attempts


In [25]:
tryout('stair', frecs)

⬜⬜🟩⬜⬜ place
🟩⬜🟨⬜⬜ skive
⬜🟨⬜⬜⬜ fidge
⬜🟨🟨⬜⬜ prink
🟩⬜🟩🟩⬜ swain
🟩🟩🟩🟩⬜ stain
Ended with a loss


#### Improving mutation
We will create a dictionary with all the words that can transformed into another allowed one by only changing one letter

In [26]:
valid_inputs = []
with open("valid-wordle-words.txt") as f:
    valid_inputs = f.read().splitlines()
comparer = set(valid_inputs)
valid_mutations = {}
alphabet = list(map(chr, range(97, 123)))
for word in valid_inputs:
    valid_mutations[word] = set()
    for i in range(len(word)):
        for letter in alphabet:
            if (letter != word[i]) and ((word[:i] + letter + word[i+1:]) in comparer):
                valid_mutations[word].add(word[:i] + letter + word[i+1:])

In [27]:
def mutate2(word, dict_frec, game_state, prob=0.03):
    if random() < prob and len(valid_mutations[word]) > 0:
        fit = 0
        res = word
        for mut in valid_mutations[word]:
            if mut in dict_frec.keys() and fitness(mut,game_state) > fit:
                fit = fitness(mut,game_state)
                res = mut
        return res
    return word

#### Improving permutation
We will now create a dictionary with all the words that can be transformd into other valid words by swapping two non-equal letters.

In [28]:
valid_inputs = []
with open("valid-wordle-words.txt") as f:
    valid_inputs = f.read().splitlines()
comparer = set(valid_inputs)
valid_permutations = {}
changes = [[i,j] for i in range(5) for j in range(i+1,5)]
for word in valid_inputs:
    valid_permutations[word] = set()
    for i,j in changes:
        swap = list(word)
        swap[i], swap[j] = swap[j], swap[i]
        swap = ''.join(swap)
        if swap != word and swap in comparer:
            valid_permutations[word].add(swap)

In [29]:
def permute2(word, dict_frec, game_state, prob=0.03):
    if random() < prob and len(valid_permutations[word]) > 0:
        fit = 0
        res = word
        for mut in valid_permutations[word]:
            if mut in dict_frec and fitness(mut,game_state) > fit:
                fit =fitness(mut,game_state)
                res = mut
        return res
    return word

#### Improving inversion
We will create a dictionary of all possible inversions of a word.

In [30]:
valid_inputs = []
with open("valid-wordle-words.txt") as f:
    valid_inputs = f.read().splitlines()
comparer = set(valid_inputs)
valid_inversions = {}
inversions = [[i,j]for i in range(4) for j in range(i+1,5)]
for word in valid_inputs:
    valid_inversions[word] = set()
    for i,j in inversions:
        inv = list(word)
        inv[i:j+1] = reversed(word[i:j+1])
        inv = ''.join(inv)
        if inv != word and inv in comparer:
            valid_inversions[word].add(inv)

In [31]:
def invert2(word, dict_frec, game_state, prob=0.02):
    if random() < prob and len(valid_inversions[word]) > 0:
        fit = 0
        res = word
        for inv in valid_inversions[word]:
            if inv in dict_frec and fitness(inv,game_state) > fit:
                fit = fitness(inv,game_state)
                res = inv
        return res
    return word

In [32]:
def wordle_genetic(game_state, guess_count, pop_size, dict_frec, max_gen, tour_size, crossover_prob,
                   mutate_prob, permute_prob, invert_prob, top_vals, top_labels):
    
    if guess_count == 0:
        guess = first_word(top_labels, top_vals)
        dict_frec.pop(guess)
        return guess
    
    possible_guesses = np.asarray(list(dict_frec.keys()))
    frecs = np.asarray(list(dict_frec.values()))

    # create population
    sample_size = min(pop_size, len(possible_guesses))
    population = np.random.choice(a = possible_guesses, size = sample_size , replace = False)
    eligible_words = set()
    generation = 0

    # do genetic iterations
    while generation < max_gen:
        # selection
        fitnesses = [fitness(p, game_state) for p in population]
        selected = selection(fitnesses, population, k=tour_size)

        # new generation
        new_pop = []
        for p1, p2 in zip(selected[0::2], selected[1::2]):
            for c in crossover(p1, p2, prob=crossover_prob):
                if (c in eligible_words) or (c not in possible_guesses):
                    new_pop.append(np.random.choice(a = possible_guesses))
                else:
                    c = mutate2(c, dict_frec, game_state, prob=mutate_prob)
                    c = permute2(c, dict_frec, game_state, prob=permute_prob)
                    c = invert2(c, dict_frec, game_state, prob=invert_prob)
                    new_pop.append(c)

        population = new_pop
        eligible_words.update(population)

        generation += 1

    # choose word in eligible_words with maximum
    best_word = min(eligible_words, key=lambda x: fitness(x, game_state))
    dict_frec.pop(best_word)
    return best_word

In [33]:
def tryout(target, frecs, crossover_prob = 0.5,  mutate_prob = 0.03, permute_prob = 0.03, invert_prob = 0.02):
    #Prep sets
    frec0 = frecs[0]
    dict_frec = frec0
    g0_sorted = sorted(frec0.items(), key=lambda x:x[1], reverse = True)
    g0_top = g0_sorted[:30]
    g0_top = np.asarray(g0_top)
    top_vals = g0_top[:,1].astype(float)
    top_vals = top_vals/top_vals.sum()
    top_labels = g0_top[:,0]
    game_state = []
    frecs = np.asarray(list(dict_frec.values()))
    frecs = (frecs + abs(min(frecs)))
    frecs = frecs/frecs.sum()
    dict_frec = dict(zip(dict_frec.keys(), frecs))
    word = wordle_genetic(game_state = game_state, guess_count = 0, pop_size = 150, 
                                         dict_frec = dict_frec, max_gen = 100, tour_size = 40,
                                        crossover_prob = crossover_prob,  mutate_prob = mutate_prob, 
                                        permute_prob = permute_prob, invert_prob = invert_prob,
                                         top_vals = top_vals, top_labels = top_labels)
    check_words(word, target)
    game_state.append((word, check_words(word, target)))
    print(str(game_state[-1][1]) + " " +game_state[-1][0])
    i = 1
    while game_state[-1][1] != "🟩🟩🟩🟩🟩" and i <= 5:
        dict_frec = update_dict(dict_frec)
        word = wordle_genetic(game_state = game_state, guess_count = i, pop_size = 150, 
                                         dict_frec = dict_frec, max_gen = 100, tour_size = 40,
                                        crossover_prob = 0.5,  mutate_prob = 0.03, permute_prob = 0.03, invert_prob = 0.02,
                                         top_vals = top_vals, top_labels = top_labels)
        check_words(word, target)
        game_state.append((word, check_words(word, target)))
        print(str(game_state[-1][1]) + " " + game_state[-1][0])
        i += 1
    if game_state[-1][1] == "🟩🟩🟩🟩🟩":
        print("Ended with a win in " + str(i) + " attempts")
    else:
        print("Ended with a loss")

In [34]:
tryout('humor', frecs, 0.5, 0.2,0.2,0.02)

⬜⬜⬜⬜🟩 never
⬜⬜⬜⬜⬜ saxes
🟨⬜⬜🟩🟩 motor
⬜🟨⬜🟨🟨 notum
🟩🟩🟩🟩🟩 humor
Ended with a win in 5 attempts


In [35]:
tryout('paint', frecs, 0.5, 0.2,0.2,0.02)

⬜🟨⬜⬜🟩 first
🟨🟨🟩⬜⬜ anise
🟨🟨⬜⬜⬜ aidas
🟩🟩🟩🟩🟩 paint
Ended with a win in 4 attempts


In [36]:
tryout('train', frecs, 0.5, 0.2,0.2,0.02)

🟨⬜⬜⬜🟨 never
🟨🟨🟨⬜⬜ anise
⬜⬜⬜🟩🟩 eloin
🟨🟨⬜🟩🟩 ramin
⬜🟩🟩🟩🟩 brain
⬜🟩🟩🟩🟩 drain
Ended with a loss


In [37]:
tryout('oscar', frecs, 1, 0.2,0.2,0.02)

🟨⬜🟨⬜⬜ state
⬜⬜⬜🟩⬜ ahead
🟨⬜🟨⬜⬜ about
⬜⬜⬜🟩⬜ litai
⬜⬜⬜🟩🟨 pujas
🟩🟩🟩🟩🟩 oscar
Ended with a win in 6 attempts


#### Improving crossover

We will create a set of all possible words made up from five letters that are allowed in wordle.

In [38]:
alphabet = np.asarray(list(map(chr, range(97, 123))))

In [39]:
def get_keys(alphabet, length):
    if len(alphabet) == 1:
        return np.asarray([alphabet[0]*length])
    elif length == 1:
        return alphabet
    else:
        res = np.asarray([])
        for i in range(len(alphabet)):
            aux = np.asarray(get_keys(alphabet[i:], length - 1))
            res = np.concatenate((res,np.char.add(np.asarray([alphabet[i]]*len(aux)), 
                                         aux)), axis = 0)
        return res
def get_words(key):
    res = set()
    p = permutations(key)
    for word in list(p):
        aux = ''.join(word)
        if aux in comparer:
            res.add(aux)
    return res

In [40]:
keys = get_keys(alphabet, 5)

In [41]:
words_from_letters = {key : get_words(key) for key in keys}

In [42]:
def crossover2(parent1, parent2, dict_prob, game_state, prob=0.5):
    child1, child2 = parent1, parent2
    if random() < prob:
        split = randrange(1, len(parent1)-1)
        child1 = parent1[:split] + parent2[split:]
        child2 = parent2[:split] + parent1[split:]
        if child1 not in dict_prob:
            key1 = ''.join(sorted(child1))
            if len(words_from_letters[key1]) > 0:
                fit = 0
                child1 = np.random.choice(a = np.asarray(list(dict_prob.keys())))
                for cand1 in words_from_letters[key1]:
                    if cand1 in dict_prob and fitness(cand1, game_state) > fit:
                        fit = fitness(cand1, game_state)
                        child1 = cand1
        if child2 not in dict_prob:
            key1 = ''.join(sorted(child2))
            if len(words_from_letters[key1]) > 0:
                fit = 0
                child2 = np.random.choice(a = np.asarray(list(dict_prob.keys())))
                for cand1 in words_from_letters[key1]:
                    if cand1 in dict_prob and fitness(cand1, game_state) > fit:
                        fit = fitness(cand1, game_state)
                        child2 = cand1
    return [child1, child2]

In [43]:
def wordle_genetic(game_state, guess_count, pop_size, dict_frec, max_gen, tour_size, crossover_prob,
                   mutate_prob, permute_prob, invert_prob, top_vals, top_labels):
    
    if guess_count == 0:
        guess = first_word(top_labels, top_vals)
        dict_frec.pop(guess)
        return guess
    
    possible_guesses = np.asarray(list(dict_frec.keys()))
    frecs = np.asarray(list(dict_frec.values()))

    # create population
    sample_size = min(pop_size, len(possible_guesses))
    population = np.random.choice(a = possible_guesses, size = sample_size , replace = False)
    eligible_words = set()
    generation = 0

    # do genetic iterations
    while generation < max_gen:
        # selection
        fitnesses = [fitness(p, game_state) for p in population]
        selected = selection(fitnesses, population, k=tour_size)

        # new generation
        new_pop = []
        for p1, p2 in zip(selected[0::2], selected[1::2]):
            for c in crossover2(p1, p2, dict_frec, game_state , prob=crossover_prob):
                if (c in eligible_words) or (c not in possible_guesses):
                    new_pop.append(np.random.choice(a = possible_guesses))
                else:
                    c = mutate2(c, dict_frec, game_state, prob=mutate_prob)
                    c = permute2(c, dict_frec, game_state, prob=permute_prob)
                    c = invert2(c, dict_frec, game_state, prob=invert_prob)
                    new_pop.append(c)

        population = new_pop
        eligible_words.update(population)

        generation += 1

    # choose word in eligible_words with maximum
    best_word = min(eligible_words, key=lambda x: fitness(x, game_state))
    dict_frec.pop(best_word)
    return best_word

In [44]:
tryout('oscar', frecs, 0.5, 0.05,0.05,0.02)

🟨⬜⬜⬜⬜ again
⬜🟨⬜⬜🟨 pooja
⬜🟨🟨🟨🟨 taros
🟩⬜⬜🟩🟩 ottar
🟩🟩🟩🟩🟩 oscar
Ended with a win in 5 attempts


In [45]:
tryout('parer', frecs, 0.5, 0.05,0.05,0.02)

⬜⬜🟨🟨⬜ there
🟨🟨⬜⬜⬜ rebid
🟩⬜⬜🟩🟩 plyer
⬜🟨⬜🟩🟩 cryer
🟩⬜⬜🟩🟩 pucer
⬜🟩⬜🟩🟩 macer
Ended with a loss


In [46]:
tryout('train', frecs, 0.5, 0.05,0.05,0.02)

⬜🟨🟨⬜🟨 first
⬜⬜🟨🟩⬜ shtik
⬜🟨⬜🟨⬜ sture
🟨🟨🟨🟩⬜ ranis
🟩🟩🟩🟩🟩 train
Ended with a win in 5 attempts


In [47]:
tryout('paint', frecs, 0.5, 0.05,0.05,0.02)

🟨⬜⬜⬜🟩 about
🟩⬜⬜⬜🟨 pooja
🟩⬜⬜🟨🟩 pleat
🟩🟨⬜⬜🟩 pilot
🟩🟩🟩🟩🟩 paint
Ended with a win in 5 attempts


### Changing the selection
We will try a roulette selection approach

In [48]:
def selection2(fitnesses, population, k=20, keep_size=True):
    props = np.asarray(fitnesses)/np.asarray(fitnesses).sum()
    return np.random.choice(a=population, p = props, replace = True, size = len(fitnesses))

In [49]:
def wordle_genetic(game_state, guess_count, pop_size, dict_frec, max_gen, tour_size, crossover_prob,
                   mutate_prob, permute_prob, invert_prob, top_vals, top_labels):
    
    if guess_count == 0:
        guess = first_word(top_labels, top_vals)
        dict_frec.pop(guess)
        return guess
    
    possible_guesses = np.asarray(list(dict_frec.keys()))
    frecs = np.asarray(list(dict_frec.values()))

    # create population
    sample_size = min(pop_size, len(possible_guesses))
    population = np.random.choice(a = possible_guesses, size = sample_size , replace = False)
    eligible_words = set()
    generation = 0

    # do genetic iterations
    while generation < max_gen:
        # selection
        fitnesses = [fitness(p, game_state) for p in population]
        selected = selection2(fitnesses, population, k=tour_size)

        # new generation
        new_pop = []
        for p1, p2 in zip(selected[0::2], selected[1::2]):
            for c in crossover2(p1, p2, dict_frec, game_state , prob=crossover_prob):
                if (c in eligible_words) or (c not in possible_guesses):
                    new_pop.append(np.random.choice(a = possible_guesses))
                else:
                    c = mutate2(c, dict_frec, game_state, prob=mutate_prob)
                    c = permute2(c, dict_frec, game_state, prob=permute_prob)
                    c = invert2(c, dict_frec, game_state, prob=invert_prob)
                    new_pop.append(c)

        population = new_pop
        eligible_words.update(population)

        generation += 1

    # choose word in eligible_words with maximum
    best_word = min(eligible_words, key=lambda x: fitness(x, game_state))
    dict_frec.pop(best_word)
    return best_word

In [50]:
tryout('train', frecs, 0.5, 0.05,0.05,0.02)

⬜⬜🟩⬜⬜ place
⬜🟩🟨🟨⬜ prink
⬜⬜🟨⬜⬜ porky
🟨🟨🟩🟨🟨 riant
🟩🟩🟩🟩🟩 train
Ended with a win in 5 attempts


In [51]:
tryout('oscar', frecs, 0.5, 0.05,0.05,0.02)

⬜⬜⬜⬜🟩 their
⬜⬜⬜⬜🟨 vinic
⬜⬜⬜🟩🟩 unbar
🟨⬜⬜🟩🟩 cymar
🟩🟩🟩🟩🟩 oscar
Ended with a win in 5 attempts


In [52]:
tryout('hello', frecs, 0.5, 0.05,0.05,0.02)

⬜⬜⬜⬜🟨 braze
⬜⬜⬜🟨⬜ pined
⬜🟨⬜⬜⬜ gobis
⬜⬜⬜⬜⬜ amici
🟨🟨⬜⬜⬜ elegy
⬜🟩⬜🟨🟨 ketol
Ended with a loss


In [53]:
tryout('death', frecs, 0.5, 0.05,0.05,0.02)

🟨🟨🟨⬜⬜ there
🟨⬜⬜🟨⬜ homer
⬜⬜⬜🟩🟩 crith
⬜🟩🟩🟩🟩 neath
⬜🟩🟩🟩🟩 beath
🟩🟩🟩🟩🟩 death
Ended with a win in 6 attempts


In [54]:
tryout('parer', frecs, 0.5, 0.05,0.05,0.02)

⬜⬜🟨⬜🟩 their
🟨⬜⬜🟩🟩 ruder
⬜🟨⬜🟨🟩 cedar
⬜⬜⬜🟩🟩 dover
⬜⬜⬜🟩🟩 ulcer
🟨🟨⬜🟩🟩 armer
Ended with a loss


In [55]:
tryout('doubt', frecs, 0.5, 0.05,0.05,0.02)

⬜🟨⬜⬜⬜ state
⬜🟩⬜⬜⬜ pooja
⬜⬜⬜⬜🟨 pined
⬜🟩🟨🟨⬜ notum
🟩🟩🟩⬜⬜ doums
🟩🟩🟩🟩🟩 doubt
Ended with a win in 6 attempts


In [56]:
tryout('trees', frecs, 0.5, 0.08,0.08,0.05)

⬜⬜⬜⬜🟨 place
⬜⬜⬜🟩⬜ unked
🟨⬜⬜🟩⬜ rifer
⬜⬜⬜🟩🟩 ishes
⬜⬜🟨🟩🟩 ogres
⬜⬜⬜🟩🟩 fomes
Ended with a loss


In [57]:
tryout('flare', frecs, 0.5, 0.05,0.05,0.02)

⬜⬜⬜⬜⬜ going
⬜🟨⬜🟨⬜ saxes
🟨⬜🟨⬜⬜ ahead
🟩🟩🟩🟩🟩 flare
Ended with a win in 4 attempts


In [58]:
tryout('panda', frecs, 0.5, 0.05,0.05,0.02)

🟨🟨⬜⬜⬜ added
🟩⬜⬜🟨⬜ pleat
⬜⬜⬜🟩⬜ elude
⬜⬜⬜🟨⬜ coxae
🟩🟩🟩🟩🟩 panda
Ended with a win in 5 attempts


In [59]:
tryout('robot', frecs, 0.5, 0.05,0.05,0.02)

⬜🟨⬜⬜⬜ state
⬜🟩🟨⬜⬜ pooja
⬜🟩🟨🟩🟨 motor
🟩🟩🟩🟩🟩 robot
Ended with a win in 4 attempts


In [71]:
tryout('flare', frecs, 0.5, 0.05,0.05,0.02)

⬜⬜🟨⬜🟨 their
🟨⬜⬜⬜🟩 anise
⬜🟨⬜⬜⬜ grips
⬜🟨⬜🟩🟩 darre
🟩🟩🟩🟩🟩 flare
Ended with a win in 5 attempts


In [72]:
tryout('proxy', frecs, 0.5, 0.05,0.05,0.02)

⬜⬜🟩⬜⬜ those
⬜⬜⬜⬜⬜ awake
⬜🟨🟨⬜⬜ typic
🟩🟩⬜⬜⬜ prism
🟩🟩🟩🟩🟩 proxy
Ended with a win in 5 attempts
